In [1]:
from nltk import tokenize
from nltk.tokenize import sent_tokenize, word_tokenize
import re
import pandas as pd
import os
from fuzzywuzzy import fuzz, process
import json

/home/jpena/jup/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## <span style = 'color:blue'>**1. Importación de posibles Diarios de Sesiones**</span>

Se abren y juntan los txt de distintos años

In [35]:
words_input_dir = '/home/jpena/1870-1891/'#Ubicación de los txt
file = []
text_str = []
for filename in os.listdir(words_input_dir):
    if filename.endswith(".txt"):
        file.append(filename)
        with open(words_input_dir + filename,'r', encoding = 'UTF-8') as input_file:
            text_str.append(input_file.read())

### <span style = 'color:blue'>**1.1. Identificación donde se menciona "asistieron los señores" y el número de sesiones**</span>

Formas típicas de inicio de asistencia según el periodo:

| Camara | Periodo                     | Frase                                                                        |
|--------|-----------------------------|------------------------------------------------------------------------------|
|   SEN  | 1870_SEN_EXT a 1881_SEN_ORD | Asistieron los señores Aldunate, Bárros, Moran...                            |
|   SEN  | 1881_SEN_EXT a 1882_SEN_ORD | Asistieron los señores:<br>Claro, Lorenzo (barra vertical) Recabárren, Manuel|
|   SEN  | 1883_SEN_ORD a 1891_SEN_ORD | Asistieron los señores:<br>Allende Padin, Ramon (tab) Pereira, Luis          |
|   DIP  | 1871_DIP_EXT a 1881_DIP_ORD |... con asistencia de los señores:<br>Amunátegui (don M.) (barra vertical) Matta (don Guillermo)| 
|   DIP  | 1882_DIP_EXT y 1882_DIP_ORD | Claro, Lorenzo (barra vertical) Recabárren, Manuel                           |
|   DIP  | 1883_DIP_EXT a 1891_DIP_ORD | Allende Padin, Ramon (tab) Pereira, Luis                                     |



In [36]:
# El output es un Dataframe resumen de todos los diarios de sesione alojados en el directorio principal
asistieron = []
for index, i in enumerate(text_str):
    asistieron.append(re.findall('asisti[eoc]ron\s*.*\s*los\s*.*\s*s[eo][ñn]ores', i))#Expresion regular
yes = []
for i in asistieron:
   yes.append(len(i))
sesiones = []
for index_s, i_s in enumerate(text_str):
    sesiones.append(re.findall('«Sesion\s[0-9]{0,2}', i_s))#Expresión regular
yes_s = []
for i_s in sesiones:
   yes_s.append(len(i_s))
data = {'name' : file, 'nro_asistieron' : yes, 'nro_sesiones' : yes_s}
df = pd.DataFrame(data, columns = ['name', 'nro_asistieron','nro_sesiones']) #df resumen, nombre del archivo, coincidencias asistieron y nro de sesiones.
df[df.nro_asistieron >= 0].head()#Se debe comprobar si en el tiff coinciden la cantidad de sesiones en el documento

,name,nro_asistieron,nro_sesiones
0,1886_DIP_ORD.txt,48,48
1,1871_DIP_ORD.txt,0,1
2,1885_DIP_EXT.txt,15,15
3,1885_DIP_ORD.txt,39,39
4,1887_DIP_ORD.txt,45,45


### <span style = 'color:blue'>**1.2. Elección y apertura del diario de sesion a procesar**</span>

In [37]:
DIP_ORD_1885_O= open(words_input_dir+'1885_DIP_ORD.txt','r', encoding = 'UTF-8')#Se elige un txt para procesarlo
DIP_ORD_1885_txt_O=DIP_ORD_1885_O.read()#Se define una variable para el diario

### <span style = 'color:blue'>**1.3. Importe de nombres y apellidos de Congresistas**</span>

In [38]:
with open('dict_parlamentarios.json') as file: #Se importa un diccionario con los nombres de los parlamentarios
    dict_parlamentarios = json.load(file)

In [12]:
#Solo en caso de requerirlo se puede importar un csv con los nombres de los Congresistas
#dfparla=pd.read_csv("Parlamentarios_BCN_1870-1891_Nombre_Apellido.csv") 
#dfparla[dfparla['Apellido_Pat']=='Freire']

In [39]:
def quitar_tildes(s):#Función para eliminar tildes
    if type(s) == type(1.0):
        return float('nan')
    s = s.replace('Á','A').replace('É','E').replace('Í','I').replace('Ó','O').replace('Ú','U')
    s = s.replace('á','a').replace('é','e').replace('í','i').replace('ó','o').replace('ú','u')
    return s

### <span style = 'color:blue'>**1.4. Creación de listas de nombres y apellidos según año**</span>

In [40]:
#Se elige el primer y segundo nombre, primer y segundo apellido, del año (Cambiar según corresponda)
Dip_1885 = [item[0]+' '+item[1]+' '+item[2]+' '+item[3] for item in list(zip(dict_parlamentarios['Diputados']['1885']['nombre_1'], dict_parlamentarios['Diputados']['1885']['nombre_2'], dict_parlamentarios['Diputados']['1885']['apellido_1'], dict_parlamentarios['Diputados']['1885']['apellido_2']))]
Dip_1885_nombre = [item.replace('Yrarrázaval', 'Irarrázaval') for item in Dip_1885] # Se reemplaza Yrarrázaval
Dip_1885_nombre = pd.Series(Dip_1885_nombre)
Dip_1885_nombre = Dip_1885_nombre.apply(lambda s: " ".join(re.split("\s+", str(s), flags=re.UNICODE)).strip())
Dip_1885_nombre = list(map(quitar_tildes, Dip_1885_nombre)) #Finalmenta se quitan las tildes y se crea en formato lista.
type(Dip_1885_nombre)

list

In [41]:
Dip_1885_nombre #El resultado es una lista con los 2 nombres y 2 apellidos de cada congresista segun el año

['Rafael Tomas Barazarte Oliva',
 'Jose Maria Cabezon Jordan',
 'Jose Ramon Segundo Rojas',
 'Abel Saavedra Ovalle',
 'Nicolas Naranjo Palacios',
 'Olegario Silva',
 'Victorino Rojas Magallanes',
 'Nicolas Gonzalez Julio',
 'Antonio Edwards Argandoña',
 'Pedro Nolasco Videla Hidalgo',
 'Joaquin Toribio Vicuña Guerrero',
 'Pedro Pablo Muñoz Godoy',
 'Francisco Puelma Tupper',
 'Jorge Segundo Huneeus Zegers',
 'Eduardo Vigil Zañartu',
 'Francisco Donoso Vergara',
 'Juan Francisco Rivas Cruz',
 'Guillermo Errazuriz Urmeneta',
 'Francisco Freire Caldera',
 'Emilio Crisologo Varas Fernandez',
 'Juan Agustin Barriga Espinosa',
 'Francisco Gandarillas Luco',
 'Francisco R. Martinez Cuadros',
 'Evaristo Marcos Sanchez Fontecilla',
 'Miguel Antonio Varas Herrera',
 'Miguel Roberto Guzman Palacios',
 'Manuel Marcial Echeverria Valdes',
 'Maximo Baltasar Sanchez Fontecilla',
 'Julio Gaete',
 'Moises David Del Fierro Arcaya',
 'Tomas Eduardo Vergara Echavarria',
 'Carlos Irarrazaval Larrain',
 'Ca

In [42]:
#Se crea una lista solo con el primer nombre del congresista
Dip_1885_1nombre =dict_parlamentarios['Diputados']['1885']['nombre_1']
Dip_1885_1nombre = list(map(quitar_tildes, Dip_1885_1nombre))
Dip_1885_1nombre

['Rafael',
 'Jose',
 'Jose',
 'Abel',
 'Nicolas',
 'Olegario',
 'Victorino',
 'Nicolas',
 'Antonio',
 'Pedro',
 'Joaquin',
 'Pedro',
 'Francisco',
 'Jorge',
 'Eduardo',
 'Francisco',
 'Juan',
 'Guillermo',
 'Francisco',
 'Emilio',
 'Juan',
 'Francisco',
 'Francisco',
 'Evaristo',
 'Miguel',
 'Miguel',
 'Manuel',
 'Maximo',
 'Julio',
 'Moises',
 'Tomas',
 'Carlos',
 'Carlos',
 'Pedro',
 'Tomas',
 'Tomas',
 'Benicio',
 'Augusto',
 'Isidoro',
 'Alberto',
 'Juan',
 'Manuel',
 'Agustin',
 'Felix',
 'Julio',
 'Jose',
 'Carlos',
 'Basilio',
 'Joaquin',
 'Eduardo',
 'Alejandro',
 'Jose',
 'Adolfo',
 'Jose',
 'Joaquin',
 'Ramon',
 'Dositeo',
 'Jose',
 'Patricio',
 'Francisco',
 'Carlos',
 'Ismael',
 'Ricardo',
 'Gonzalo',
 'Federico',
 'Pedro',
 'Jose',
 'Dario',
 'Eduardo',
 'Jose',
 'Lauro',
 'Gaspar',
 'Luis',
 'Jose',
 'Juan',
 'Anibal',
 'Juan',
 'Manuel',
 'Jose',
 'Miguel',
 'Benjamin',
 'Francisco',
 'Manuel',
 'Ramon',
 'Federico',
 'Rafael',
 'Juan',
 'Estanislao',
 'Jose',
 'Ricardo'

In [43]:
#Se crea una lista solo con el primer apellido del congresista
Dip_1885_1apellido =dict_parlamentarios['Diputados']['1885']['apellido_1']
Dip_1885_1apellido = list(map(quitar_tildes, Dip_1885_1apellido))
Dip_1885_1apellido

['Barazarte',
 'Cabezon',
 'Segundo',
 'Saavedra',
 'Naranjo',
 'Silva',
 'Rojas',
 'Gonzalez',
 'Edwards',
 'Videla',
 'Vicuña',
 'Muñoz',
 'Puelma',
 'Huneeus',
 'Vigil',
 'Donoso',
 'Rivas',
 'Errazuriz',
 'Freire',
 'Varas',
 'Barriga',
 'Gandarillas',
 'Martinez',
 'Sanchez',
 'Varas',
 'Guzman',
 'Echeverria',
 'Sanchez',
 'Gaete',
 'Del Fierro',
 'Vergara',
 'Yrarrazaval',
 'Valenzuela',
 'Montt',
 'Echavarria',
 'Torres',
 'Alamos',
 'Matte',
 'Errazuriz',
 'Edwards',
 'Mackenna',
 'Vicuña',
 'Edwards',
 'Echeverria',
 'Prieto',
 'Waddington',
 'Eastman',
 'Soffia',
 'Castro',
 'Matte',
 'Fierro',
 'Balmaceda',
 'Murillo',
 'Aguirre',
 'Rodriguez',
 'Barros',
 'Errazuriz',
 'Hurtado de Mendoza',
 'Lynch',
 'Valdes',
 'Lira',
 'Tocornal',
 'Perez',
 'Bulnes',
 'Errazuriz',
 'Lagos',
 'Letelier',
 'Sanchez',
 'De la Barra',
 'Balmaceda',
 'Barros',
 'Toro',
 'Aldunate',
 'Valderrama',
 'Parga',
 'Zañartu',
 'Echaurren',
 'Valenzuela',
 'Zegers',
 'Cruz',
 'Lavin',
 'Valdes',
 'So

In [44]:
'Valdes' in Dip_1885_1apellido #busqueda de posible nombre en la lista

True

## <span style = 'color:blue'>**2. Tokenización del texto**</span>

Tokeniza por oraciones, en este caso se utiliza el archivo segun corresponda .txt. Se devuelve una lista de token

In [45]:
Token_sent=sent_tokenize(DIP_ORD_1885_txt_O,'spanish')
type(Token_sent)

list

In [22]:
#En caso de requerirlo, se puede tokenizar todos los textos de la variable text_str (No lo recomiendo)
#for i in text_str:
#    Token_sent_todos=sent_tokenize(i,'spanish')
#len(Token_sent_todos)

### <span style = 'color:blue'>**2.1. Coincidencias de expresión regular en cada token**</span>

Se buscan coincidencias de 'asistieron los señores' en cada token de la lista Token_sent y se devuelve el nro del índice

In [46]:
Token_sent_index = [i for i, item in enumerate(Token_sent) if re.search('asisti[eoc]ron\s*.*\s*los\s*.*\s*s[eo][ñn]ores', item)]
len(Token_sent_index) #Token_sent_index devuelve el número del indice de cada coincidencia

39

### <span style = 'color:blue'>**2.3. Extracción de bloques**</span>

In [47]:
#Se utiliza el indice con la frase asistieron los señores y se extrae el bloque donde debería estar cada asistente
Tokens_bruto_bloque=[]
for i in Token_sent_index:
    Tokens_bruto_bloque.append(Token_sent[i-5:i+5])#+- 5 token desde la coincidencia con la expresión regular
bloque=[]
for i in Tokens_bruto_bloque:
    asis=' '.join(map(str, i))
    bloque.append(asis)
    bloque_txt=[]
    for sp in bloque:
        asis2=sp.split('\n')
        bloque_txt.append(asis2) #Se devuelve una lista con sublistas de cada sesión

In [48]:
#Se juntan las 'x' listas en una sola
listado_bloquetxt = []
for i in bloque_txt:
    for j in i:
        listado_bloquetxt.append(j)
#print(listado_bloquetxt)

## <span style = 'color:blue'>**3. Creación y limpieza de SeriesDataframe**</span>

In [49]:
#Se crea un pandas Series con la lista de sesiones para posterior limpieza
se_listado_bloquetxt = pd.Series(listado_bloquetxt)
se_listado_bloquetxt.head(50)

0                                              * n DA 1
1                                            4,9 mn 2 "
2                                            5.9 m 43 "
3     Con esto, i habiendo el señor Presidente recomen-
4     dado a los miembros de las indicadas comisione...
5     despacharan los respectivos informes para la p...
6     sesion ordinaria, i a todos los señores Diputa...
7     so sirvieran hacer tomar nota en Secretaría de...
8     micilios, se levantó la sesion a la 1 h. 50 ms...
9                                                      
10    «Sesion 27. * estraordinaria en 20 de enero de...
11    Presidencia del señor Lastarria. —Se abrió a l...
12                  ms. P. M, i asistieron los señores:
13                                   Aldunate, Federico
14                               Balmaceda, José Manuel
15                                Balmaceda, José Marta
16                                Barriga, Juan Agustin
17                                    Barazarte,

### <span style = 'color:blue'>**3.1. Limpieza de texto**</span>

In [50]:
#Se limpia el texto, se eliminan caracteres especiales, se eliminan espación adicionales, se liminan nan y se reinicio el indice
se_listado_bloquetxt=se_listado_bloquetxt.apply(lambda s: re.sub('[^a-zA-Zá-úÁ-Ú0-9,«]',' ', str(s)).strip(' ')).copy()
se_listado_bloquetxt=se_listado_bloquetxt.apply(quitar_tildes)
se_listado_bloquetxt=se_listado_bloquetxt.apply(lambda s: " ".join(re.split("\s+", str(s), flags=re.UNICODE)).strip(','))
se_listado_bloquetxt = se_listado_bloquetxt.dropna()
se_listado_bloquetxt = se_listado_bloquetxt.reset_index (drop = True)
se_listado_bloquetxt.head()

0                                               n DA 1
1                                             4,9 mn 2
2                                             5 9 m 43
3     Con esto, i habiendo el señor Presidente recomen
4    dado a los miembros de las indicadas comisione...
dtype: object

In [51]:
#Se eliminan '' y False
se_listado_bloquetxt2=[x for x in se_listado_bloquetxt if x != '']
lista_asistentes=[x for x in se_listado_bloquetxt2 if x != False]
type(lista_asistentes)

list

In [52]:
#La lista se vuelve a convertir en un SerieDataframe
serie_sesiones = pd.Series(lista_asistentes)
serie_sesiones.head()

0                                               n DA 1
1                                             4,9 mn 2
2                                             5 9 m 43
3     Con esto, i habiendo el señor Presidente recomen
4    dado a los miembros de las indicadas comisione...
dtype: object

## <span style = 'color:blue'>**4. Identificación nro de sesiones y Congresistas**</span>

In [53]:
%%time 
def nro_sesiones(s):#Función busca el nro de la sesion en la Serie
    sesiones=re.findall('Sesion\s[0-9]{0,2}', s)
    if sesiones == []:
        return None
    else:
        return " ".join(sesiones)

CPU times: user 6 µs, sys: 1 µs, total: 7 µs
Wall time: 16.9 µs


In [54]:
%%time
Se_nro_sesiones=serie_sesiones.apply(nro_sesiones)
Se_nro_sesiones=Se_nro_sesiones.fillna(method = 'ffill')#Se crea una Serie con el numero de sesión. Este nro repite hasta encontrar el próximo nro
df_nro_sesiones=pd.DataFrame({'Texto':serie_sesiones, 'Nro_sesion': Se_nro_sesiones})#Se crea un df la columna texto y Nro_sesion
print(df_nro_sesiones)

                                                  Texto Nro_sesion
0                                                n DA 1       None
1                                              4,9 mn 2       None
2                                              5 9 m 43       None
3      Con esto, i habiendo el señor Presidente recomen       None
4     dado a los miembros de las indicadas comisione...       None
...                                                 ...        ...
4138  el cuarto, a la de Guerra ei quinto i ala de L...  Sesion 38
4139  cion i Justicia el sesto 3 7 De un informe de ...  Sesion 38
4140  Justicia, favorable, en jeneral, a la mocion d...  Sesion 38
4141  Montt don Pedro, relativa a establecer en las ...  Sesion 38
4142  el rejimen de separacion individual Se mando p...  Sesion 38

[4143 rows x 2 columns]
CPU times: user 11.5 ms, sys: 0 ns, total: 11.5 ms
Wall time: 10.4 ms


In [36]:
#Posible búsqueda por nro de sesión
#df_nro_sesiones[df_nro_sesiones['Nro_sesion']=='Sesion 39']

### <span style = 'color:blue'>**4.1. Identificación Congresista**</span>

In [55]:
%%time
def congresistas(s):#Función para identificar congresistas según el largo del texto en cada fila
    ssplit=s.split(' ')
    if 1 < len(ssplit) <= 6:
        return s
    else:
        return False

CPU times: user 6 µs, sys: 2 µs, total: 8 µs
Wall time: 16.7 µs


In [56]:
df_nro_sesiones.loc[:,'Congresistas']=df_nro_sesiones['Texto'].apply(congresistas)#Se aplica función congresistas
df_nro_sesiones

,Texto,Nro_sesion,Congresistas
0,n DA 1,None,n DA 1
1,"4,9 mn 2",None,"4,9 mn 2"
2,5 9 m 43,None,5 9 m 43
3,"Con esto, i habiendo el señor Presidente recomen",None,False
4,dado a los miembros de las indicadas comisione...,None,False
...,...,...,...
4138,"el cuarto, a la de Guerra ei quinto i ala de L...",Sesion 38,False
4139,cion i Justicia el sesto 3 7 De un informe de ...,Sesion 38,False
4140,"Justicia, favorable, en jeneral, a la mocion d...",Sesion 38,False
4141,"Montt don Pedro, relativa a establecer en las ...",Sesion 38,False


In [57]:
df_nro_sesiones = df_nro_sesiones.drop(df_nro_sesiones[df_nro_sesiones['Congresistas']==False].index)#Se eliminan los False
df_nro_sesiones = df_nro_sesiones.reset_index(drop=True)#Se reinicia indice
df_nro_sesiones

,Texto,Nro_sesion,Congresistas
0,n DA 1,None,n DA 1
1,"4,9 mn 2",None,"4,9 mn 2"
2,5 9 m 43,None,5 9 m 43
3,"Aldunate, Federico",Sesion 27,"Aldunate, Federico"
4,"Balmaceda, Jose Manuel",Sesion 27,"Balmaceda, Jose Manuel"
...,...,...,...
2828,"Zegers, Julio",Sesion 38,"Zegers, Julio"
2829,i el Secretario,Sesion 38,i el Secretario
2830,dio cuenta,Sesion 38,dio cuenta
2831,"departamentos de Putaendo, Santiago, Cachapoal",Sesion 38,"departamentos de Putaendo, Santiago, Cachapoal"


### <span style = 'color:blue'>**4.2. Filtro de Congresistas**</span>

In [30]:
%%time 
#Función para filtrar congresistas
def congresistas_filtro(s): #Se compararan dos listas para encontrar un 'nombre1' o apellido1, sino devuelve un False
    ssplit=s.split(' ')
    comparacion = [item for item in ssplit if item in Dip_1885_1nombre]#Se utiliza un listado solo de nombres
    if len(comparacion) > 0:
        return (s)
    else:
        ssplit=s.split(',')
        comparacion2 = [item for item in ssplit if item in Dip_1885_1apellido]#Se utiliza un listado solo de appellido
        if len(comparacion2) > 0:
            return (s)
        else:
            return (False)

CPU times: user 7 µs, sys: 2 µs, total: 9 µs
Wall time: 17.4 µs


In [58]:
%%time
df_nro_sesiones.loc[:,'Congresistas']=df_nro_sesiones['Congresistas'].apply(lambda s: congresistas_filtro(s))#Se aplica función en columna congresistas
df_nro_sesiones

CPU times: user 31.6 ms, sys: 0 ns, total: 31.6 ms
Wall time: 30.2 ms


,Texto,Nro_sesion,Congresistas
0,n DA 1,None,False
1,"4,9 mn 2",None,False
2,5 9 m 43,None,False
3,"Aldunate, Federico",Sesion 27,"Aldunate, Federico"
4,"Balmaceda, Jose Manuel",Sesion 27,"Balmaceda, Jose Manuel"
...,...,...,...
2828,"Zegers, Julio",Sesion 38,"Zegers, Julio"
2829,i el Secretario,Sesion 38,False
2830,dio cuenta,Sesion 38,False
2831,"departamentos de Putaendo, Santiago, Cachapoal",Sesion 38,False


In [59]:
#Revisar Falsos y verificar si hay nombres mal escritos para reemplazar
#df_nro_sesiones.loc[df_nro_sesiones.loc[:, 'Congresistas']== False].head(60)
#df_nro_sesiones.loc[df_nro_sesiones.loc[:, 'Congresistas']== False][0:50]
len(df_nro_sesiones.loc[df_nro_sesiones.loc[:, 'Congresistas']== False])

253

In [60]:
#Se reemplazan nombres mal escritos en columna texto
df_nro_sesiones=df_nro_sesiones.replace({'Freire, Zenon':'Juan Zenon Freire',
                                         'Bañados E , Kaon':'Ramon Bañados Espinosa',
                                        'Urrutia, Gregerio':'Urrutia, Gregorio',
                                         'Barazarte, Rafazel':'Barazarte, Rafael',
                                         'Prieto, Mannel A':'Prieto, Manuel A',
                                        'Saavedra, Ahel':'Abel Saavedra Ovalle',
                                         'Orrego Luco':'Orrego Luco, Augusto',
                                         'Barros, Latro':'Barros, Lauro',
                                         'Barros Luco, Raiten':'Barros Luco, Ramon',
                                         'Konig, Abrahain':'Konig, Abraham',
                                         'Rosas, Pantalcon':'Rosas, Pantaleon',
                                         'Zañartu, Anmibal':'Zañartu, Anibal',
                                         'Sanhueza Lizardi, Rafuel':'Sanhueza Lizardi, Rafael',
                                         'Tocornal, Emrique':'Tocornal, Enrique',
                                         'Lira, Carios':'Lira, Carlos',
                                         'Vidal, Cabriel':'Vidal, Gabriel',
                                         'Walker Martinez, Cirlos':'Walker Martinez, Carlos',
                                         'Yavar, Rumon':'Yavar, Ramon',
                                         'Zañartu, Amibal':'Zañartu, Anibal',
                                         'Valdes Valdes, Ismacl':'Valdes Valdes, Ismael	',
                                         'Barros Luco, Kamon':'Barros Luco, Ramon',
                                         'Montt Albano, Rafacl':'Montt Albano, Rafael',
                                         'Valdes Valdes, Tsmael':'Valdes Valdes, Ismael',
                                         'Carvallo Elizalde, Trancisco':'Carvallo Elizalde, Francisco',
                                         'Vidal, Cabricl':'Vidal, Gabriel',
                                         'Bernales, Ramen':'Bernales, Ramon',
                                         'Cruchaga, Mignel':'Cruchaga, Miguel',
                                         'Freiro, Zenon':'Juan Zenon Freire',
                                         'Tocornal, Enrigne':'Tocornal, Enrique',
                                         'Fernandez Albano, Elfias':'Fernandez Albano, Elias',
                                         'Montt Albano, Rafacl':'Montt Albano, Rafael',
                                         'Guerrero, Acolfo':'Guerrero, Adolfo',
                                         'Walker Martinez, Carios':'Walker Martinez, Carlos',
                                         'Balbontin, Manel G':'Balbontin, Manuel G',
                                         'Vial Bello, Iduardo':'Vial Bello, Eduardo',
                                         'Rodriguez Ojeda, Ambrosic':'Rodriguez Ojeda, Ambrosio',
                                         'Rogers, Curlos':'Rogers, Carlos',
                                         'Huneens, Jorje':'Huneens, Jorge',
                                         'Montt, Petro':'Montt, Pedro'})
df_nro_sesiones.loc[df_nro_sesiones.loc[:, 'Congresistas']== False]
#df_nro_sesiones.loc[df_nro_sesiones.loc[:, 'Asistentes']== False][50:100]
#df_nro_sesiones.head(65)

,Texto,Nro_sesion,Congresistas
0,n DA 1,None,False
1,"4,9 mn 2",None,False
2,5 9 m 43,None,False
37,el señor Ministro de Guer,Sesion 27,False
38,rai Marmai el secretario,Sesion 27,False
...,...,...,...
2777,"Huneens, Jorge",Sesion 38,False
2800,"Rodriguez Ojeda, Ambrosio",Sesion 38,False
2829,i el Secretario,Sesion 38,False
2830,dio cuenta,Sesion 38,False


In [61]:
%%time
#Se aplica nuevamente la función congresistas_filtro, si fuese necesario
df_nro_sesiones.loc[:,'Congresistas']=df_nro_sesiones['Texto'].apply(congresistas_filtro)
df_nro_sesiones

CPU times: user 31.1 ms, sys: 496 µs, total: 31.6 ms
Wall time: 29.8 ms


,Texto,Nro_sesion,Congresistas
0,n DA 1,None,False
1,"4,9 mn 2",None,False
2,5 9 m 43,None,False
3,"Aldunate, Federico",Sesion 27,"Aldunate, Federico"
4,"Balmaceda, Jose Manuel",Sesion 27,"Balmaceda, Jose Manuel"
...,...,...,...
2828,"Zegers, Julio",Sesion 38,"Zegers, Julio"
2829,i el Secretario,Sesion 38,False
2830,dio cuenta,Sesion 38,False
2831,"departamentos de Putaendo, Santiago, Cachapoal",Sesion 38,False


In [62]:
#Se eliminan False y reinicia indice
#len(df_nro_sesiones.loc[df_nro_sesiones.loc[:, 'Congresistas']== False])
df_nro_sesiones = df_nro_sesiones.drop(df_nro_sesiones[df_nro_sesiones['Congresistas']==False].index)#Se eliminan los False
df_nro_sesiones = df_nro_sesiones.reset_index(drop=True)#Se reinicia indice
df_nro_sesiones

,Texto,Nro_sesion,Congresistas
0,"Aldunate, Federico",Sesion 27,"Aldunate, Federico"
1,"Balmaceda, Jose Manuel",Sesion 27,"Balmaceda, Jose Manuel"
2,"Balmaceda, Jose Marta",Sesion 27,"Balmaceda, Jose Marta"
3,"Barriga, Juan Agustin",Sesion 27,"Barriga, Juan Agustin"
4,"Barazarte, Rafael",Sesion 27,"Barazarte, Rafael"
...,...,...,...
2594,"Walker Martinez, Carlos",Sesion 38,"Walker Martinez, Carlos"
2595,"Yavar, Ramon",Sesion 38,"Yavar, Ramon"
2596,"Zañartu, Manuel Aristides",Sesion 38,"Zañartu, Manuel Aristides"
2597,"Zegers, Julio",Sesion 38,"Zegers, Julio"


### <span style = 'color:blue'>**4.3. Ordenar nombre de Congresistas**</span>

In [63]:
%%time
def orden_nombre(s):#Función para ordenar los nombres en columna Congresistas (Nombre apellido)
    if s == False:
        return False
    else:
        ssplit=s.split(',')
        if len(ssplit)<= 1:
            return s
        elif len(ssplit) == 2:
            return(ssplit[1]+' '+ssplit[0])
        else:
            return False

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 15 µs


In [64]:
df_nro_sesiones.loc[:,'Congresistas_ord']=df_nro_sesiones['Congresistas'].apply(orden_nombre)
df_nro_sesiones.head()

,Texto,Nro_sesion,Congresistas,Congresistas_ord
0,"Aldunate, Federico",Sesion 27,"Aldunate, Federico",Federico Aldunate
1,"Balmaceda, Jose Manuel",Sesion 27,"Balmaceda, Jose Manuel",Jose Manuel Balmaceda
2,"Balmaceda, Jose Marta",Sesion 27,"Balmaceda, Jose Marta",Jose Marta Balmaceda
3,"Barriga, Juan Agustin",Sesion 27,"Barriga, Juan Agustin",Juan Agustin Barriga
4,"Barazarte, Rafael",Sesion 27,"Barazarte, Rafael",Rafael Barazarte


In [65]:
#Se revisa si hay algún False en columna Congresistas_ord
df_nro_sesiones.loc[df_nro_sesiones.loc[:, 'Congresistas_ord']== False]

,Texto,Nro_sesion,Congresistas,Congresistas_ord


In [ ]:
#Se eliminan False si es que existieran
#df_nro_sesiones = df_nro_sesiones.drop(df_nro_sesiones[df_nro_sesiones['Congresistas_ord']==False].index)# Se eliminan False
#df_nro_sesiones = df_nro_sesiones.reset_index(drop=True)#Se reinicia indice
#df_nro_sesiones

## <span style = 'color:blue'>**5. Listado de asistentes por sesión**</span>

In [66]:
%%time
def asistentes(s):#Función para encontrar nombres completos de congresistas
    q=process.extractOne(s,Dip_1885_nombre,scorer = fuzz.QRatio)
    w=process.extractOne(s,Dip_1885_nombre,scorer = fuzz.WRatio)
    if q[1]>=87:
        return (q[0])
    else:
        if w[1]>=87:
            return (w[0])
        else:
            return (s)

CPU times: user 6 µs, sys: 1e+03 ns, total: 7 µs
Wall time: 15.7 µs


In [67]:
%%time
df_nro_sesiones.loc[:,'Asistentes']=df_nro_sesiones['Congresistas_ord'].apply(lambda s:asistentes(s))#Se crea columna Asistentes
df_nro_sesiones.head()

CPU times: user 7min 33s, sys: 0 ns, total: 7min 33s
Wall time: 7min 33s


,Texto,Nro_sesion,Congresistas,Congresistas_ord,Asistentes
0,"Aldunate, Federico",Sesion 27,"Aldunate, Federico",Federico Aldunate,Federico Aldunate Palacios
1,"Balmaceda, Jose Manuel",Sesion 27,"Balmaceda, Jose Manuel",Jose Manuel Balmaceda,Jose Manuel Balmaceda Fernandez
2,"Balmaceda, Jose Marta",Sesion 27,"Balmaceda, Jose Marta",Jose Marta Balmaceda,Jose Marta Balmaceda
3,"Barriga, Juan Agustin",Sesion 27,"Barriga, Juan Agustin",Juan Agustin Barriga,Juan Agustin Barriga Espinosa
4,"Barazarte, Rafael",Sesion 27,"Barazarte, Rafael",Rafael Barazarte,Rafael Barazarte


## <span style = 'color:blue'>**6. DataFrame final**</span>

Se eliminan columnas y incluye el nombre del diario(txt) al DataFrame

In [81]:
df_nro_sesiones = df_nro_sesiones.drop(columns=['Texto','Congresistas','Congresistas_ord'])#Se eliminan columnas
df_nro_sesiones['doc']='1885_DIP_EXT' #Se incluye el nombre del diario
df_nro_sesiones['año'] = df_nro_sesiones['doc'].apply(lambda s: s.split('_')[0])#Se extrae el año del nombre del txt.
df_nro_sesiones['camara'] = df_nro_sesiones['doc'].apply(lambda s: s.split('_')[1])#Se extrae el tip de camara del nombre del txt.
df_nro_sesiones['tipo'] = df_nro_sesiones['doc'].apply(lambda s: s.split('_')[2])#Se extrae eltipo de sesión del nombre del txt.

In [82]:
df_nro_sesiones1

,Nro_sesion,Asistentes,doc,año,camara,tipo
0,Sesion 27,Federico Aldunate Palacios,1885_DIP_EXT,1885,DIP,EXT
1,Sesion 27,Jose Manuel Balmaceda Fernandez,1885_DIP_EXT,1885,DIP,EXT
2,Sesion 27,Jose Marta Balmaceda,1885_DIP_EXT,1885,DIP,EXT
3,Sesion 27,Juan Agustin Barriga Espinosa,1885_DIP_EXT,1885,DIP,EXT
4,Sesion 27,Rafael Barazarte,1885_DIP_EXT,1885,DIP,EXT
...,...,...,...,...,...,...
2594,Sesion 38,Carlos Walker Martinez,1885_DIP_EXT,1885,DIP,EXT
2595,Sesion 38,Jose Ramon Yavar Jimenez,1885_DIP_EXT,1885,DIP,EXT
2596,Sesion 38,Manuel Aristides Zañartu Zañartu,1885_DIP_EXT,1885,DIP,EXT
2597,Sesion 38,Julio Zegers Samaniego,1885_DIP_EXT,1885,DIP,EXT


In [71]:
#se reordenan las columnas, 'doc', 'año', 'camara', 'tipo','Nro_sesion', 'Asistentes'
cols = df_nro_sesiones.columns.tolist()
cols = cols[-4:]+cols[:2]
df_nro_sesiones=df_nro_sesiones[cols]
df_nro_sesiones['tipo'].loc[0:82]='ORD'#Se cambia el tipo de la sesion 27

/home/jpena/jup/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [72]:
#Se deben concatenar todos los años y sesiones
df_nro_sesiones

,doc,año,camara,tipo,Nro_sesion,Asistentes
0,1885_DIP_EXT,1885,DIP,ORD,Sesion 27,Federico Aldunate Palacios
1,1885_DIP_EXT,1885,DIP,ORD,Sesion 27,Jose Manuel Balmaceda Fernandez
2,1885_DIP_EXT,1885,DIP,ORD,Sesion 27,Jose Marta Balmaceda
3,1885_DIP_EXT,1885,DIP,ORD,Sesion 27,Juan Agustin Barriga Espinosa
4,1885_DIP_EXT,1885,DIP,ORD,Sesion 27,Rafael Barazarte
...,...,...,...,...,...,...
2594,1885_DIP_EXT,1885,DIP,EXT,Sesion 38,Carlos Walker Martinez
2595,1885_DIP_EXT,1885,DIP,EXT,Sesion 38,Jose Ramon Yavar Jimenez
2596,1885_DIP_EXT,1885,DIP,EXT,Sesion 38,Manuel Aristides Zañartu Zañartu
2597,1885_DIP_EXT,1885,DIP,EXT,Sesion 38,Julio Zegers Samaniego


In [73]:
#Se exporta en un archivo csv
#df_nro_sesiones.to_csv('Listado_asistencia.csv', index = False, encoding = 'utf-8')